## Data Reading

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricks1ete.dfs.core.windows.net/orders")
df.count()
display(df)

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed('_rescued_data', 'rescued_data')

In [0]:
df = df.drop('rescued_data')
display(df)

In [0]:
df = df.withColumn('order_date', to_timestamp(col('order_date')))

display(df)

In [0]:

df  = df.withColumn("year", year(col("order_date")))
display(df)

In [0]:
df1 = df.withColumn("flag" , dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

display(df1)

In [0]:
df1 = df1.withColumn("rank_flag" , rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

display(df1)

In [0]:
df1 = df1.withColumn('row_flag', row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

display(df1)

### Classes - OOP


In [0]:
class window:
    def dense_rank(self,df ):
        df_dense_rank = df.withColumn("dense_flag" , dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    def rank(self,df):
         df_rank = df.withColumn("rank_flag" , dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
         return df_rank
    def row_number(self,df):
        df_row_rank = df.withColumn("row_flag" , dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_rank

In [0]:
df_new = df;
display(df_new)

In [0]:
obj = window()

In [0]:
df_result = obj.dense_rank(df_new)
display(df_result)

### Data Writing

In [0]:
df.write.format("delta").mode("append").save("abfss://silver@databricks1ete.dfs.core.windows.net/orders")